In [1]:
import pandas as pd
import numpy as np

'''
Load Data
'''
IDIR = 'G://bigdata//badou//00-data//'
priors = pd.read_csv(IDIR + 'order_products__prior.csv', dtype={
    'order_id': np.int32,
    'product_id': np.int16,
    'add_to_cart_order': np.int16,
    'reordered': np.int8})
train = pd.read_csv(IDIR + 'order_products__train.csv', dtype={
    'order_id': np.int32,
    'product_id': np.int16,
    'add_to_cart_order': np.int16,
    'reordered': np.int8})
orders = pd.read_csv(IDIR + 'orders.csv', dtype={
    'order_id': np.int32,
    'user_id': np.int32,
    'eval_set': 'object',
    'order_number': np.int16,
    'order_dow': np.int8,
    'order_hour_of_day': np.int8,
    'days_since_prior_order': np.float32})
products = pd.read_csv(IDIR + 'products.csv', dtype={
    'product_id': np.int16,
    'order_id': np.int32,
    'aisle_id': np.int8,
    'department_id': np.int8}, usecols=['product_id', 'aisle_id', 'department_id'])

In [2]:
print('priors {}: {}'.format(priors.shape, ', '.join(priors.columns)))
print('orders {}: {}'.format(orders.shape, ', '.join(orders.columns)))
print('train {}: {}'.format(train.shape, ', '.join(train.columns)))

priors (32434489, 4): order_id, product_id, add_to_cart_order, reordered
orders (3421083, 7): order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order
train (1384617, 4): order_id, product_id, add_to_cart_order, reordered


In [7]:
# 1. product feature
print('computing product f')
prods = pd.DataFrame()
# 产品销量 int32
prods['orders'] = priors.groupby('product_id').size().astype(np.int32)

computing product f


In [4]:
priors.product_id.head(10)

0   -32416
1    28985
2     9327
3   -19618
4    30035
5    17794
6   -25395
7     1819
8   -21868
9   -31782
Name: product_id, dtype: int16

In [8]:
prods.head(10)

,orders
product_id,
-32768,502
-32767,7
-32766,14
-32765,485
-32764,110
-32763,587
-32762,5
-32761,263
-32760,232


In [9]:
priors.product_id

0          -32416
1           28985
2            9327
3          -19618
4           30035
            ...  
32434484   -25858
32434485    11352
32434486     4600
32434487    24852
32434488     5020
Name: product_id, Length: 32434489, dtype: int16

In [10]:
priors.head(
)

,order_id,product_id,add_to_cart_order,reordered
0,2,-32416,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,-19618,4,1
4,2,30035,5,0


In [11]:
priors2 = pd.read_csv(IDIR + 'order_products__prior.csv', dtype={
    'order_id': str,
    'product_id': str,
    'add_to_cart_order': np.int16,
    'reordered': np.int8})

In [12]:
priors2.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [13]:
prods['reorders'] = priors['reordered'].groupby(priors.product_id).sum().astype(np.float32)

In [14]:
prods.head()

,orders,reorders
product_id,,
-32768,502,208.0
-32767,7,2.0
-32766,14,3.0
-32765,485,335.0
-32764,110,54.0


In [15]:
prods['reorder_rate'] = (prods.reorders / prods.orders).astype(np.float32)

In [16]:
products = products.join(prods, on='product_id')

In [17]:
products.head()

,product_id,aisle_id,department_id,orders,reorders,reorder_rate
0,1,61,19,1852.0,1136.0,0.613391
1,2,104,13,90.0,12.0,0.133333
2,3,94,7,277.0,203.0,0.732852
3,4,38,1,329.0,147.0,0.446809
4,5,5,13,15.0,9.0,0.600000


In [18]:
products.set_index('product_id', drop=False, inplace=True)

In [19]:
products.head()

,product_id,aisle_id,department_id,orders,reorders,reorder_rate
product_id,,,,,,
1,1,61,19,1852.0,1136.0,0.613391
2,2,104,13,90.0,12.0,0.133333
3,3,94,7,277.0,203.0,0.732852
4,4,38,1,329.0,147.0,0.446809
5,5,5,13,15.0,9.0,0.600000


In [20]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [21]:
orders.set_index('order_id', inplace=True, drop=False)

In [22]:
priors.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,-32416,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,-19618,4,1
4,2,30035,5,0


In [23]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
order_id,,,,,,,
2539329,2539329,1,prior,1,2,8,NaN
2398795,2398795,1,prior,2,3,7,15.0
473747,473747,1,prior,3,3,12,21.0
2254736,2254736,1,prior,4,4,7,29.0
431534,431534,1,prior,5,4,15,28.0


In [24]:
priors = priors.join(orders, on='order_id', rsuffix='_')

In [25]:
priors.head()

,order_id,product_id,add_to_cart_order,reordered,order_id_,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,-32416,1,1,2,202279,prior,3,5,9,8.0
1,2,28985,2,1,2,202279,prior,3,5,9,8.0
2,2,9327,3,0,2,202279,prior,3,5,9,8.0
3,2,-19618,4,1,2,202279,prior,3,5,9,8.0
4,2,30035,5,0,2,202279,prior,3,5,9,8.0
